# Linear Model MCMC
Part of the Bayesian neural networks via MCMC: a Python-based tutorial

This section of the tutorial covers the development of an MCMC algorithm applied to a simple linear model.

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import numpy as np
import pandas as pd
from ipywidgets import interact, fixed, widgets
# visulisation function
sys.path.append('/project')
from functions.visualisations import (
    histogram_trace, plot_linear_data,
    plot_y_timeseries, boxplot_weights,
    plot_ycorr_scatter
)
from tqdm import tqdm
from types import MethodType

np.random.seed(2023)

In [3]:
os.chdir('/project')

## Define a class with the functions and attributes required for a linear model

- `predict`: Function to output y given the input data and model parameters - $y = w x + b$
- `evaluate_proposal`: Function to load a given proposal distribution ($\theta$) and return the model prediction
- `encode`: Helper function to encode the model parameters ($\theta$) into the model as $w$ and $b$

In [4]:
from models.linear_model import LinearModel
from models.mcmc import MCMC_Linear as MCMC

## Load the data
- Load in the suspot data
- You can also load in the other regeression datasets `Lazer` and `Energy`

In [6]:
# load the data
name        = "Iris"
train_data   = np.loadtxt("data/{}/train.txt".format(name))
test_data    = np.loadtxt("data/{}/test.txt".format(name))

print('Training data shape: {}'.format(train_data.shape))

Training data shape: (105, 5)


## Sample using MCMC

- Create the MCMC loop and sample the posterior distribution

In [7]:
n_samples = 1100000 # number of samples to draw from the posterior
burn_in = 100000 # number of samples to discard before recording draws from the posterior

thin_factor = 100

# or load from sunspot data
x_data = train_data[:,:-1]
y_data = train_data[:,-1]
x_test = test_data[:,:-1]
y_test = test_data[:,-1]

if name in ['Sunspot','Abalone']:
    layer_sizes = [x_data.shape[1], 1]
    data_case = 'regression'
elif name in ['Iris']:
    layer_sizes = [x_data.shape[1], 3]
    data_case = 'classification'
elif name in ['Ionosphere']:
    layer_sizes = [x_data.shape[1], 2]
    data_case = 'classification'
else:
    raise ValueError('data_case is invalid.')

thinned_res = []

## MCMC Settings and Setup
for this_chain in np.arange(5):
    print('Chain: {}'.format(this_chain))
    np.random.seed(2023 + this_chain)
    # Initialise the MCMC class
    lm = LinearModel(layer_sizes=layer_sizes,data_case=data_case)
    mcmc = MCMC(lm,n_samples, burn_in, x_data, y_data, x_test, y_test)
    # Run the sampler
    results, pred = mcmc.sampler()
    # thin to remove autocorrelation and to reduce the size of the data 
    results = results.iloc[::thin_factor,:]
    for _ in pred.keys():
        pred[_] = pred[_][::thin_factor,:]

    thinned_res.append(results)

Chain: 0


KeyboardInterrupt: 

In [ ]:
# gather the predicitons into useful variables
pred_y = pred['train_pred']
sim_y = pred['train_sim']
pred_y_test = pred['test_pred']
sim_y_test = pred['test_sim']

In [ ]:
thinned_res[0].shape

conc_res = np.stack(thinned_res,axis=0)
# results = pd.concat(thinned_res, axis=0)

In [ ]:
print(conc_res.shape)

from convergence.convergence import gelman_rubin
# run gelman rubin test
gr = gelman_rubin(conc_res)

# print the results into pandas then print
gr_df = pd.DataFrame(data = gr, index=thinned_res[0].columns).T
gr_df.index = ['Rhat']
gr_df

## Analyse the results
Plot the data with the mean linear fit and some uncertainty.

Plot the posterior distribution and trace for each parameter using ipywidgets.

In [ ]:
# Print the train/test RMSE
trainRMSE = np.array([mcmc.rmse(pred_y[_,:], y_data) for _ in np.arange(pred_y.shape[0])])
testRMSE = np.array([mcmc.rmse(pred_y_test[_,:], y_test) for _ in np.arange(pred_y_test.shape[0])])

print('Train RMSE: {:.5f} ({:.5f})'.format(trainRMSE.mean(),trainRMSE.std()))
print('Test RMSE: {:.5f} ({:.5f})'.format(testRMSE.mean(),testRMSE.std()))

In [ ]:
def plot_hist(results, param_name):
    # results = results_rmse
    posterior_mean = results[param_name].mean()
    print('{:.3f} mean value of posterior'.format(posterior_mean))
    histogram_trace(results[param_name].values)

# use ipywidgets to get a "gui" dropdown to view all the parameters
interact(
    plot_hist, 
    results=fixed(results), 
    param_name=widgets.Dropdown(
        options=results.columns,
        value='w0',
        description='Parameter:',
    )
)
None